In [2]:
import geopandas as gpd
import os, glob
import xarray as xr
from osgeo import gdal
from osgeo import ogr
import numpy as np
import rasterio as rio
import pandas as pd
from shapely import geometry
from shapely.geometry import mapping
import shapely.geometry
import rioxarray as rxr
import pandas as pd
os.chdir("/home/btufail@eurac.edu/Documents/SRR3_notebooks/")

In [16]:
eusalpbb = gpd.read_file("notebooks/resources/UC8/Local/alpinespace_eusalp_boundingbox.shp")
eusalpbb_laea = eusalpbb.to_crs('EPSG:3035')

In [18]:
# Reproject to projected coordinate system
#eusalpbb = eusalpbb.to_crs('EPSG:3857')
total_bounds = eusalpbb_laea.total_bounds
minX, minY, maxX, maxY = total_bounds
x, y = (minX, minY)
geom_array = []
square_size = 75000
while y < maxY:
    while x < maxX:
        geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
        geom_array.append(geom)
        x += square_size
    x = minX
    y += square_size
 
eusalpbb_sites = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:3035')
eusalpbb_sites.insert(0, 'ID', range(0, 0 + len(eusalpbb_sites)))

In [13]:
eusalpbb_sites.to_file('eusalpbb.geojson')

In [19]:
crlc = rxr.open_rasterio("/mnt/CEPH_PROJECTS/sao/ForestCanopy/01_Data/CORINE_2018_250m_Raster/DATA/U2018_CLC2018_V2020_20u1.tif", masked=True,)

In [20]:
clc_classes = pd.read_csv("notebooks/resources/UC8/Local/clc_classes_final.csv")

In [32]:
hrl_join = pd.read_csv("notebooks/resources/UC8/Local/hrl_table_geo.csv")

In [28]:
geo_grid = []
for i in range(0, len(hrl_join)):
    r_grid = (hrl_join['Grid'][i])
    pre, ext = os.path.splitext(r_grid)
    geo = pre+'.geojson'
    geo_grid.append(geo)
hrl_join['geo_grid'] = geo_grid  


In [34]:
for i in range(0, len(hrl_join)):
    data = hrl_join['TCD'][i]
    output = hrl_join["Tile"][i]+ '_grid.geojson'
    ra = rio.open(data)
    bounds  = ra.bounds
    minX, minY, maxX, maxY = np.array(bounds)
    geom_array = []
    x, y = (minX, minY)
    square_size = 400
    while y < maxY:
        while x < maxX:
            geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
            geom_array.append(geom)
            x += square_size
        x = minX
        y += square_size
    fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:3035')
    fishnet.to_file(output, driver='GeoJSON')
    break

In [ ]:
for i in range(0, len(hrl_join)):
    r1 = rxr.open_rasterio(hrl_join['TCD'][i]) 
    r2 = rxr.open_rasterio(hrl_join['DLT'][i])
    r_grid = gpd.read_file(hrl_join['geo_grid'][i])
    result = calcMetric(r1,r2,crlc,r_grid)
    result_gpd = gpd.GeoDataFrame(result, columns=['geometry', 'DomAbs', 'DomType', 'HRLPerc', 'Density', 'CLCclasses', 'CLCperc', 'CLCother']).set_crs('EPSG:3035')
    outfile = hrl_join["Tile"][i]+ '_shapefile_select_noscore.geojson'
    result_gpd.to_file(outfile, driver='GeoJSON')
    break

In [14]:
def calcMetric(r1,r2,r3,j):
    b1 = pd.DataFrame(columns = ['geometry', 'DomAbs', 'DomType', 'HRLPerc', 'Density', 'CLCclasses', 'CLCperc', 'CLCother']) 
    for index, row in j.iterrows():
        #print(row)
        cr_tcd = r1.rio.clip(row)
        cr_trtype = r2.rio.clip(row)
        cr_crlc = r3.rio.clip(row, from_disk=True)
    
    #Calculate tree density#
        cr_tcd_mn = int(np.mean(cr_tcd))
        if cr_tcd_mn > 90 or cr_tcd_mn < 10:
            continue
        
        cr_trtype_b = cr_trtype[0]
        unique, counts = np.unique(cr_trtype_b, return_counts=True)
        tab = np.asarray((unique, counts)).T
        tab = np.sort(tab)[::-1]
        if len(tab) <2:
            continue 
        brd = int(sum(tab[1]))/int(sum(sum(tab)))
        conf = int(sum(tab[0]))/int(sum(sum(tab)))
        dominance = (brd-conf)/(brd+conf)
        dominant_type = "BroadLeaved" if dominance > 0 else "Coniferous"
        dominance_abs = round(abs(dominance)*100)
        forest_perc = round((brd+conf)*100)
        if forest_perc > 90 or forest_perc < 10:
            continue
        
        cr_crlc = cr_crlc[0]
        unique, counts = np.unique(cr_crlc, return_counts=True)
        #print(unique, counts)
        dataset = pd.DataFrame({'Raster_ID': unique, 'n': counts})
        dataset_m = dataset.merge(clc_classes, on='Raster_ID', how='left')
        dataset_m = dataset_m[['L1_class', 'Target' , 'n']]
        if dataset_m.isin(['Forests']).any().any():
            if len(dataset_m.index) == 1:
                pass
            pass
        clc_nclasses = len(dataset_m.index)
        clc_forest = round(((dataset_m.loc[dataset_m['Target'] =="Forests", 'n'])/dataset['n'].sum())*100).tolist()
        clc_other_d = dataset_m.loc[dataset_m['Target'] !="Forests"]
        clc_other_ds = dataset_m.sort_values(by=['n'])
        clc_other = dataset_m['Target'].values[0]
        #if clc_forest > 90 or clc_forest < 10:
        #    pass
        if clc_nclasses == 5 or clc_nclasses < 2:
            pass
        data = []
        data = pd.DataFrame({'geometry': row.geometry,'DomAbs': dominance_abs, 'DomType': dominant_type, 'HRLPerc': forest_perc, 'Density': cr_tcd_mn, 'CLCclasses': clc_nclasses, 'CLCperc': clc_forest, 'CLCother': clc_other})
        b1 = b1.append(data)
    return(b1)
    

In [11]:
#Tree density and forest percentage
Values = list(range(0, 101))
Scores = np.concatenate([([i]*10) for i in [1,2,3,4,5,5,4,3,2,1]], axis=0)
Scores = np.insert(Scores, 0, 0)
score_density = pd.DataFrame({'Values': Values, 'Scores': Scores})

In [12]:
# CLC Classes
Values = list(range(0, 11))
Scores = [1,5,5,3,1,0,0,0,0,0]
Scores = np.insert(Scores, 0, 0)
score_nclc = pd.DataFrame({'Values': Values, 'Scores': Scores})

In [13]:
# Tree Dominance
Values = list(range(0, 101))
Scores = np.concatenate([([i]*20) for i in [1,2,3,4,5]], axis=0)
Scores = np.insert(Scores, 0, 0)
score_dom = pd.DataFrame({'Values': Values, 'Scores': Scores})

In [14]:
data_noscore = []
for file in sorted(glob.glob('/home/btufail@eurac.edu/Documents/SRR3_notebooks/AllShapes/*.geojson')):
    data_noscore.append(file)


In [15]:
all = pd.DataFrame(columns = ['geometry', 'DomAbs', 'DomType', 'HRLPerc', 'Density', 'CLCclasses', 'CLCperc', 'CLCother', 'Scoredom', 'ScorePerc1', 'ScorePerc2', 'ScoreCLC', 'ScoreDens', 'ScoreAll', 'inTile'])
for file in data_noscore:
    check = gpd.read_file(file)
    poly_2 = []
    intile=[]
    for index, row in check.iterrows():
        poly = row['geometry']
        for index1, tile in eusalpbb_sites.iterrows():
            tile_poly = tile['geometry']
            within = tile_poly.contains(poly)
            if within:
                intile.append(tile['ID'])
                poly_2.append(row)
            else:
                pass
    poly_2 = pd.DataFrame(poly_2)
    DomAbs_list = poly_2.DomAbs.tolist()
    Scoredom = [int(score_dom.loc[score_dom['Values'] == int(i), 'Scores']) for i in DomAbs_list]
    HRLPerc_list = poly_2.HRLPerc.tolist()
    ScorePerc1 = [int(score_density.loc[score_density['Values'] == int(i), 'Scores']) for i in HRLPerc_list]
    CLCperc_list = poly_2.CLCperc.tolist()
    ScorePerc2 = [int(score_density.loc[score_density['Values'] == int(i), 'Scores']) for i in CLCperc_list]
    CLCclasses_list = poly_2.CLCclasses.tolist()
    ScoreCLC = [int(score_nclc.loc[score_nclc['Values'] == int(i), 'Scores']) for i in CLCclasses_list]
    Density_list = poly_2.Density.tolist()
    ScoreDens = [int(score_density.loc[score_density['Values'] == int(i), 'Scores']) for i in Density_list]
    ScoreAll = []
    for x in range (0, len (Scoredom)):  
        ScoreAll.append(Scoredom[x] + ScorePerc1[x] + ScorePerc2[x] + ScoreCLC[x] + ScoreDens[x])  
    polyscore_2 = poly_2.assign(Scoredom=Scoredom, ScorePerc1=ScorePerc1, ScorePerc2=ScorePerc2, ScoreCLC=ScoreCLC, ScoreDens=ScoreDens, ScoreAll=ScoreAll, inTile=intile)
    
    all = all.append(polyscore_2, ignore_index=True)
comb = gpd.GeoDataFrame(all, columns=['geometry', 'DomAbs', 'DomType', 'HRLPerc', 'Density', 'CLCclasses', 'CLCperc', 'CLCother', 'Scoredom', 'ScorePerc1', 'ScorePerc2', 'ScoreCLC', 'ScoreDens', 'ScoreAll', 'inTile']).set_crs('EPSG:3035')

In [17]:
unique, counts = np.unique((comb['inTile']), return_counts=True)
comb_bytile = pd.DataFrame({'ID': unique,'Polygons': counts})

In [18]:
polybytile = eusalpbb_sites.merge(comb_bytile, on='ID', how='left')
polybytile = polybytile.dropna()
polybytile = polybytile.sort_values(by=['Polygons'], ascending=False)
selectedTiles = polybytile.head(150)

In [93]:
all2 = pd.DataFrame(columns = ['geometry', 'DomAbs', 'DomType', 'HRLPerc', 'Density', 'CLCclasses', 'CLCperc', 'CLCother', 'Scoredom', 'ScorePerc1', 'ScorePerc2', 'ScoreCLC', 'ScoreDens', 'ScoreAll', 'inTile'])
for index, row in selectedTiles.iterrows():
    sel = comb.loc[comb['inTile'] == index]
    sel2 = sel.sort_values(by=['ScoreAll'], ascending=False)
    sel3 = sel2.loc[sel2['ScoreAll'] == max(sel2['ScoreAll'])]
    sel4 = sel3.sort_values(by=['inTile'])
    all2 = all2.append(sel4)
comb2 = gpd.GeoDataFrame(all2, columns=['geometry', 'DomAbs', 'DomType', 'HRLPerc', 'Density', 'CLCclasses', 'CLCperc', 'CLCother', 'Scoredom', 'ScorePerc1', 'ScorePerc2', 'ScoreCLC', 'ScoreDens', 'ScoreAll', 'inTile']).set_crs('EPSG:3035')

In [130]:
all3 = pd.DataFrame(columns = ['geometry', 'DomAbs', 'DomType', 'HRLPerc', 'Density', 'CLCclasses', 'CLCperc', 'CLCother', 'Scoredom', 'ScorePerc1', 'ScorePerc2', 'ScoreCLC', 'ScoreDens', 'ScoreAll', 'inTile'])
for i in comb2['inTile'].unique():
    tosample = comb2.loc[comb2['inTile'] == i]
    smp = tosample.sample(1)
    all3 = all3.append(smp)
comb3 = gpd.GeoDataFrame(all3, columns=['geometry', 'DomAbs', 'DomType', 'HRLPerc', 'Density', 'CLCclasses', 'CLCperc', 'CLCother', 'Scoredom', 'ScorePerc1', 'ScorePerc2', 'ScoreCLC', 'ScoreDens', 'ScoreAll', 'inTile']).set_crs('EPSG:3035')